In [1]:
import matplotlib.pyplot as plt
import pandas as pd
from processing import biosppy_processing, nk_processing, processing_aggregate
from time_domain import time
from frequency_domain import frequency
from nonlinear import nonlinear
import numpy as np

from tqdm.notebook import tqdm

## 0. Read Data

In [2]:
sampling_rate = 2000
raw = pd.read_csv('./data/ecg_70.txt', sep = ' ').T
raw

,1,2,3,4,5,6,7,8,9,10,...,239991,239992,239993,239994,239995,239996,239997,239998,239999,240000
id1,0.794525,0.790710,0.785828,0.781708,0.776520,0.772095,0.767212,0.762329,0.757141,0.752411,...,1.750950,1.688230,1.624760,1.560060,1.494600,1.428530,1.362300,1.295780,1.230010,1.164400
id2,0.033112,0.032196,0.031433,0.030670,0.030060,0.029144,0.028381,0.027771,0.027466,0.026703,...,-0.015564,-0.015564,-0.015869,-0.016022,-0.016327,-0.017090,-0.017700,-0.017853,-0.018616,-0.019226
id3,0.039368,0.036316,0.033875,0.031738,0.030518,0.029602,0.029144,0.029297,0.030365,0.031281,...,0.131683,0.130920,0.130463,0.130005,0.129852,0.129700,0.129547,0.129700,0.129852,0.129395
id4,-0.361328,-0.355072,-0.348663,-0.342712,-0.336456,-0.330963,-0.325470,-0.320129,-0.315094,-0.310364,...,-0.195160,-0.194702,-0.193939,-0.193634,-0.192261,-0.191803,-0.191193,-0.190735,-0.189972,-0.189514
id5,-0.017853,-0.016937,-0.016480,-0.015869,-0.015411,-0.014954,-0.014496,-0.014343,-0.014343,-0.014038,...,0.103607,0.079346,0.055542,0.033417,0.011444,-0.009155,-0.029144,-0.048065,-0.066223,-0.082703
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
id69,0.095825,0.095825,0.095825,0.095673,0.095520,0.095215,0.095215,0.094910,0.094910,0.095062,...,0.083008,0.083618,0.083313,0.083771,0.083771,0.084228,0.084381,0.084686,0.084839,0.084839
id70,0.001526,0.002594,0.003662,0.004578,0.005798,0.006714,0.008240,0.009308,0.011139,0.012665,...,-0.070953,-0.088043,-0.103302,-0.116577,-0.128784,-0.139008,-0.148010,-0.155334,-0.161133,-0.166016
id71,-0.545654,-0.547791,-0.549164,-0.550690,-0.551758,-0.553131,-0.554047,-0.555115,-0.555878,-0.556793,...,0.080261,0.080414,0.080566,0.080566,0.080261,0.079803,0.079803,0.079651,0.079346,0.079346
id72,-0.001984,-0.003815,-0.004730,-0.006561,-0.007935,-0.009613,-0.011444,-0.013275,-0.014648,-0.016937,...,-0.074463,-0.074768,-0.074768,-0.074768,-0.074616,-0.074921,-0.074921,-0.074921,-0.075226,-0.075378


## 1. Time Domain

In [3]:
time_nk_result = dict()
for row in tqdm(raw.iterrows(), total = len(raw)):
    id = row[0]
    sampling_rate = 2000
    signal = row[1].tolist()
    pro = nk_processing(signal, sampling_rate)
    time_nk_result[id] = time(pro['rpeaks'], 2000)
    
    
time_biosppy_result = dict()
for row in tqdm(raw.iterrows(), total = len(raw)):
    id = row[0]
    sampling_rate = 2000
    signal = row[1].tolist()
    pro = biosppy_processing(signal, sampling_rate)
    time_biosppy_result[id] = time(pro['rpeaks'], 2000)
    
    
time_aggregate_result = dict()
for row in tqdm(raw.iterrows(), total = len(raw)):
    id = row[0]
    sampling_rate = 2000
    signal = row[1].tolist()
    pro = processing_aggregate(signal, sampling_rate)
    time_aggregate_result[id] = time(pro['rpeaks'], 2000)


  0%|          | 0/70 [00:00<?, ?it/s]

  0%|          | 0/70 [00:00<?, ?it/s]

  0%|          | 0/70 [00:00<?, ?it/s]

In [8]:
pd.set_option('display.float_format', lambda x: f'{x:.2f}') 

In [27]:
temp = time_nk_result
temp = pd.DataFrame(temp).T.describe().loc[['mean', 'std']].T
temp[temp.index.str.endswith('rmssd')]



,mean,std
pyhrv rmssd,46.39,36.54
nk rmssd,46.39,36.54


In [28]:
temp = time_biosppy_result
temp = pd.DataFrame(temp).T.describe().loc[['mean', 'std']].T
temp[temp.index.str.endswith('rmssd')]



,mean,std
pyhrv rmssd,45.56,32.62
nk rmssd,45.56,32.62


In [29]:
temp = time_aggregate_result
temp = pd.DataFrame(temp).T.describe().loc[['mean', 'std']].T
temp[temp.index.str.endswith('rmssd')]



,mean,std
pyhrv rmssd,46.38,36.53
nk rmssd,46.38,36.53


## 2. Frequency Domain

In [4]:
frequency_nk_result = dict()
for row in tqdm(raw.iterrows(), total = len(raw)):
    id = row[0]
    sampling_rate = 2000
    signal = row[1].tolist()
    pro = nk_processing(signal, sampling_rate)
    frequency_nk_result_ = dict()
    output = frequency(pro['rpeaks'], 2000)
    frequency_nk_result_['pyhrv welch'] = output['pyhrv welch']['fft_ratio']
    frequency_nk_result_['pyhrv ar'] = output['pyhrv ar']['ar_ratio']
    frequency_nk_result_['nk welch'] = output['nk welch']['HRV_LFHF']
    frequency_nk_result_['nk fft'] = output['nk fft']['HRV_LFHF']
    frequency_nk_result[id] = frequency_nk_result_



frequency_biosppy_result = dict()
for row in tqdm(raw.iterrows(), total = len(raw)):
    id = row[0]
    sampling_rate = 2000
    signal = row[1].tolist()
    pro = biosppy_processing(signal, sampling_rate)
    frequency_biosppy_result_ = dict()
    output = frequency(pro['rpeaks'], 2000)
    frequency_biosppy_result_['pyhrv welch'] = output['pyhrv welch']['fft_ratio']
    frequency_biosppy_result_['pyhrv ar'] = output['pyhrv ar']['ar_ratio']
    frequency_biosppy_result_['nk welch'] = output['nk welch']['HRV_LFHF']
    frequency_biosppy_result_['nk fft'] = output['nk fft']['HRV_LFHF']
    frequency_biosppy_result[id] = frequency_biosppy_result_




frequency_aggregate_result = dict()
for row in tqdm(raw.iterrows(), total = len(raw)):
    id = row[0]
    sampling_rate = 2000
    signal = row[1].tolist()
    pro = processing_aggregate(signal, sampling_rate)
    frequency_aggregate_result_ = dict()
    output = frequency(pro['rpeaks'], 2000)
    frequency_aggregate_result_['pyhrv welch'] = output['pyhrv welch']['fft_ratio']
    frequency_aggregate_result_['pyhrv ar'] = output['pyhrv ar']['ar_ratio']
    frequency_aggregate_result_['nk welch'] = output['nk welch']['HRV_LFHF']
    frequency_aggregate_result_['nk fft'] = output['nk fft']['HRV_LFHF']
    frequency_aggregate_result[id] = frequency_aggregate_result_





  0%|          | 0/70 [00:00<?, ?it/s]

  0%|          | 0/70 [00:00<?, ?it/s]

  0%|          | 0/70 [00:00<?, ?it/s]

In [9]:
temp = frequency_nk_result
temp = pd.DataFrame(temp).T.describe().loc[['mean', 'std']].T
temp



,mean,std
pyhrv welch,2.28,1.97
pyhrv ar,1.85,0.06
nk welch,2.25,1.84
nk fft,2.26,1.75


In [10]:
temp = frequency_biosppy_result
temp = pd.DataFrame(temp).T.describe().loc[['mean', 'std']].T
temp



,mean,std
pyhrv welch,2.23,2.01
pyhrv ar,1.85,0.06
nk welch,2.21,1.90
nk fft,2.22,1.77


In [11]:
temp = frequency_aggregate_result
temp = pd.DataFrame(temp).T.describe().loc[['mean', 'std']].T
temp



,mean,std
pyhrv welch,2.28,1.97
pyhrv ar,1.85,0.06
nk welch,2.23,1.80
nk fft,2.24,1.73


## 3. Nonlienar

In [13]:
nonlinear_nk_result = dict()
for row in tqdm(raw.iterrows(), total = len(raw)):
    id = row[0]
    sampling_rate = 2000
    signal = row[1].tolist()
    pro = nk_processing(signal, sampling_rate)
    nonlinear_nk_result_ = dict()
    output = nonlinear(pro['rpeaks'], 2000)
    nonlinear_nk_result_['pyhrv'] = output['pyhrv nonlinear']['sd1/sd2']
    nonlinear_nk_result_['nk'] = output['nk nonlinear']['HRV_SD1SD2']
    nonlinear_nk_result[id] = nonlinear_nk_result_


nonlinear_biosppy_result = dict()
for row in tqdm(raw.iterrows(), total = len(raw)):
    id = row[0]
    sampling_rate = 2000
    signal = row[1].tolist()
    pro = biosppy_processing(signal, sampling_rate)
    nonlinear_biosppy_result_ = dict()
    output = nonlinear(pro['rpeaks'], 2000)
    nonlinear_biosppy_result_['pyhrv'] = output['pyhrv nonlinear']['sd1/sd2']
    nonlinear_biosppy_result_['nk'] = output['nk nonlinear']['HRV_SD1SD2']
    nonlinear_biosppy_result[id] = nonlinear_biosppy_result_


nonlinear_aggregate_result = dict()
for row in tqdm(raw.iterrows(), total = len(raw)):
    id = row[0]
    sampling_rate = 2000
    signal = row[1].tolist()
    pro = processing_aggregate(signal, sampling_rate)
    nonlinear_aggregate_result_ = dict()
    output = nonlinear(pro['rpeaks'], 2000)
    nonlinear_aggregate_result_['pyhrv'] = output['pyhrv nonlinear']['sd1/sd2']
    nonlinear_aggregate_result_['nk'] = output['nk nonlinear']['HRV_SD1SD2']
    nonlinear_aggregate_result[id] = nonlinear_aggregate_result_




  0%|          | 0/70 [00:00<?, ?it/s]

  0%|          | 0/70 [00:00<?, ?it/s]

  0%|          | 0/70 [00:00<?, ?it/s]

In [14]:
temp = nonlinear_nk_result
temp = pd.DataFrame(temp).T.describe().loc[['mean', 'std']].T
temp



,mean,std
pyhrv,0.43,0.21
nk,0.43,0.21


In [15]:
temp = nonlinear_biosppy_result
temp = pd.DataFrame(temp).T.describe().loc[['mean', 'std']].T
temp



,mean,std
pyhrv,0.43,0.22
nk,0.43,0.22


In [16]:
temp = nonlinear_aggregate_result
temp = pd.DataFrame(temp).T.describe().loc[['mean', 'std']].T
temp



,mean,std
pyhrv,0.43,0.21
nk,0.43,0.21
